In [3]:
import pandas as pd
from selenium import webdriver
import time as t
from bs4 import BeautifulSoup
import numpy as np
import requests


In [4]:
def arrumar_link(acao, empresa):
    ''' 
        Função destinada a retornar se a ação é odinaria ou preferencial
        e fazendo o concatenação na função main
        Parametros:
        acao -> Tikcker 
        empresa -> nome da empresa na B3
        tipo -> on-ordinaria , pn-preferencial, unit-on e pn
        retorno -> string concatenada com variaveis empresa, acao e tipo
    '''
    
    # se nome de acao o final for 3 faça tipo recebe on #
    # se final do nome da acao for 4 tipo recebe pn #
    # se final de acao for igual a 11 então tipo recebe unit #
    if acao[-1:] == '3':
        tipo = 'on'
    elif acao[-1:] == '4' or acao[-1:]=='5' or acao[-1:]== '6':
        tipo = 'pn'
    elif acao[-2:] == '11':
        tipo = 'unit'
    # se tipo  for igual a unit então retorna sem o tipo #
    # caso contrario retorna empresa tipo e acao #
    if tipo == 'unit':
        return f'{empresa}-{acao}'
    else:
        return f'{empresa}-{tipo}-{acao}'
        

In [5]:
def retorna_int_href(acao, empresa):
    from time import sleep
    from selenium.webdriver.chrome.options import Options
    ''' 
        Função que retorna o numero de interações em que o for vai ser feito para a ação especifica
        comeco -> inteiro 
        final -> inteiro
        a -> string -> variavel que recebe o ajuste da função link 
        html -> Seletor 
        bs -> lista -> recebe todo o html
        linhas -> string -> aonde se encontra a estring que tem o valor maximo de tabelas para a interação
    '''
    comeco = 0
    final = 0
    a = arrumar_link(acao, empresa)
    url = f'https://br.advfn.com/bolsa-de-valores/bovespa/{a}/historico/mais-dados-historicos?current=0&Date1=03/02/17&Date2=04/05/22'
    option = Options()
    option.headless = True
    browser = webdriver.Chrome(options=option)
    sleep(2)
    browser.get(url)
    sleep(2)
    html = browser.page_source
    sleep(2)
    browser.close()
    bs = BeautifulSoup(html, 'html.parser')
    
    linhas = bs.find_all('a')[40]['href']
    # for dentro da string #
    # se linhas tiver '=' na string então comeco recebe o numero do array mais 1 #
    # se linhas na posição i  for igual a & então final recebe i que é o valor do array na posição aonde se encontra &#
    for i in range(len(linhas)):
        if linhas[i]== '=' :
            comeco = i+1
        elif linhas[i] == '&':
            final = i
            break
    # mostrar o link em linhas que nos intereça #
    #print(linhas)
    return int(linhas[comeco:final])

In [6]:
def adjustando_dataFrame(df):
    from datetime import datetime as dt
    from tqdm import tqdm
    import warnings
    warnings.filterwarnings("ignore")
    
    ''' 
        df -> Dataframe  aonde tem todos os valores puxados do webscraping 
        df['Fechamento'] -> valor de fechamento da acao
    '''
    
    df['Fechamento'] = df['Fechamento'].map(lambda t: t.replace(',', '.'))
    df['Fechamento'] = df['Fechamento'].map(lambda t: float(t))
    df['Variacao'] = df['Variacao'].map(lambda t: t.replace(",", "."))
    df['Variacao'] = df['Variacao'].map(lambda t: float(t))
    df['Variacao_%'] = df['Variacao_%'].map(lambda t: t.replace("%", ""))
    df['Variacao_%'] = df['Variacao_%'].map(lambda t: t.replace(",", "."))
    df['Variacao_%'] = df['Variacao_%'].map(lambda t: float(t))
    df['Abertura'] = df['Abertura'].map(lambda t: t.replace(",", "."))
    df['Abertura'] = df['Abertura'].map(lambda t: float(t))
    df['Maxima'] = df['Maxima'].map(lambda t: t.replace(",", "."))
    df['Maxima'] = df['Maxima'].map(lambda t: float(t))
    df['Minima'] = df['Minima'].map(lambda t: t.replace(",", "."))
    df['Minima'] = df['Minima'].map(lambda t: float(t))
    
    # transforma a data em string e arruma aonde tem Jan para /01/ etc ...#
    #df['Data'] = df['Data'].map(lambda x: str(x))
    df['Data'] = df['Data'].map(lambda x: x.replace('Jan', '/01/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Fev', '/02/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Mar', '/03/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Abr', '/04/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Mai', '/05/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Jun', '/06/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Jul', '/07/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Ago', '/08/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Set', '/09/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Out', '/10/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Nov', '/11/'))
    df['Data'] = df['Data'].map(lambda x: x.replace('Dez', '/12/'))
    # for  que intera dentro de data e altera todos os valores  para datas # 
    for i in tqdm(range(len(df['Data']))):
        df['Data'][i] = dt.strptime(df['Data'][i], '%d /%m/ %Y').date()  
    # arruma os valores em crescente no dataframe em relação a data fechamento, variacao, variacao a porcentagem, ... #    
    df = df.sort_values(by=['Data','Fechamento', 'Variacao', 'Variacao_%', 'Abertura', 'Maxima', 'Minima'])    
    # indexa a data como o index do dataframe #
    df = df.set_index('Data', drop=True)
    return df

In [7]:
def import_test(acao, empresa):
    from tqdm import tqdm
    from pandas import DataFrame
    from selenium import webdriver
    from time import sleep
    from bs4 import BeautifulSoup
    from selenium.webdriver.chrome.options import Options
    ''' 
        acao -> string -> acao digitada pelo usuario
        empresa -> string -> nome da empresa
        num -> int -> inteiro que é recebido pela funçao retorna_int
        a -> string -> recebe uma 
        data -> lista -> que recebe os textos da string html 
        datanew -> lista -> que organiza limpa os dados da lista anterior 
        Days, Close, Variation, Varietion_p, Open, Max, Min -> lista -> que separa cada valor do webscraping
        df -> Dataframe -> aonde recebe as listas 
    '''
    Days, Close, Variation, Variation_p, Open, Max, Min = [],[],[],[],[],[],[]
    empresa = empresa.strip().lower().replace(' ', '-')
    a = arrumar_link(acao, empresa)
    num = retorna_int_href(acao, empresa)
    for i in tqdm(range(num, -1, -1)):
        url2 = f'http://br.advfn.com/bolsa-de-valores/bovespa/{a}/historico/mais-dados-historicos?current={i}&Date1=28/01/17&Date2=28/04/22'
        sleep(2)
        option = Options()
        sleep(2)
        option.headless = True
        sleep(2)
        browser = webdriver.Chrome(options=option)
        sleep(2)
        browser.get(url2)
        sleep(2)
        html = browser.page_source
        sleep(2)
        browser.close()
        bs = BeautifulSoup(html, 'html.parser')
        linhas = bs.find_all('tr', attrs={'class': 'result'})
        data = []
        j = 0
        while j < len(linhas):
            data.append(linhas[j].text)
            j += 1
        k = 0
        datanew = []
        while k < len(data):
            datanew.append(data[k].split('\n'))
            k += 1
        l = 1
        while l < len(datanew):
            Days.append(datanew[l][1])
            Close.append(datanew[l][2])
            Variation.append(datanew[l][3])
            Variation_p.append(datanew[l][4])
            Open.append(datanew[l][5])
            Max.append(datanew[l][6])
            Min.append(datanew[l][7])
            l += 1
            
    df = DataFrame({
            'Data': Days,
            'Fechamento': Close,
            'Variacao': Variation,
            'Variacao_%': Variation_p,
            'Abertura': Open,
            'Maxima': Max,
            'Minima': Min,
    })
    
    df = adjustando_dataFrame(df)
    
    df.to_csv(f'{acao}-{empresa}.csv')
    
    return df
        

In [8]:
acao = 'YDUQ3'
empresa = 'estacio part'

In [13]:
df2 = import_test(acao, empresa)

100%|██████████| 730/730 [00:00<00:00, 2033.64it/s]


In [ ]:
def df_adjust(df):
    indice = list()
    for i in range(len(df)):
        if len(df['Unnamed: 1'][i]==5):
            if df['Unnamed: 1'][i][-1:]=='3' or df['Unnamed: 1'][i][-1:]=='5' or df['Unnamed: 1'][i][-1:]=='6':
                pass
            else:
                indice.append(i)
        elif len(df['Unnamed: 1'][i])== 6:
            if df['Unnamed: 1'][i][-2:] !='11':
                indice.append(i)
        else:
            indice.append(i)
    df.drop(indice, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

------

In [92]:
def import_adfn_acoes_empresas():
    lista_abc = [chr(i).upper() for i in range(ord('a'), ord('z')+1)]


    from time import sleep
    from selenium.webdriver.chrome.options import Options
    from tqdm import tqdm
    nome_empresa =[]
    nome_acao = []
    for i in tqdm(range(len(lista_abc))):
        url_abc = f'https://br.advfn.com/bolsa-de-valores/bovespa/{lista_abc[i]}'
        
        option = Options()
        option.headless = True
        browser = webdriver.Chrome(options=option)
        sleep(2)
        browser.get(url_abc)
        sleep(2)
        html = browser.page_source
        sleep(2)
        browser.close()
        bs = BeautifulSoup(html, 'html.parser')
        linhas = bs.find_all('table', attrs={'class': 'atoz-link-bov'})
        sleep(2)
        df = pd.read_html(str(linhas))[0]
        df = df_adjust(df)
        nome_empresa.append(df.iloc[:,-2].to_list())
        nome_acao.append(df.iloc[:, -1].to_list())
    return nome_acao, nome_empresa

    

In [ ]:
nome_acao, nome_empresa = import_adfn_acoes_empresas()

In [ ]:
df = pd.read_html(str(linhas))[0]

In [ ]:
for j in range(len(nome_acao)):
    for i in range(len(nome_acao[j])):
        print(nome_empresa[j][i])
        print(nome_acao[j][i])